In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "data_load", ENTITY_NAME: "boston_dataset" },
    ],
    outputs = 
    [
        { ENTITY_NAME: "df_X_train" },
        { ENTITY_NAME: "df_Y_train" },
        { ENTITY_NAME: "df_X_eval" },
        { ENTITY_NAME: "df_Y_eval" },
        { ENTITY_NAME: "df_X_test" },
        { ENTITY_NAME: "df_Y_test" },
        
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
#5 read inputs 
prev_step_inputs = substep.inputs(step_name="data_load")

boston_df = spark.read.parquet(prev_step_inputs.boston_dataset)

In [ ]:
#6 prepare the data and divide the boston dataset into train, eval and test subsets
from sklearn.model_selection import train_test_split

train_df, test_df = boston_df.randomSplit(weights=[0.7,0.3], seed=239)
eval_df, test_df = test_df.randomSplit(weights=[0.5,0.5], seed=239)

x_train = train_df.drop("MEDV")
y_train = train_df.select("MEDV")

x_eval = eval_df.drop("MEDV")
y_eval = eval_df.select("MEDV")

x_test = test_df.drop("MEDV")
y_test = test_df.select("MEDV")

In [ ]:
#7 write outputs
outputs = substep.outputs()

x_train.write.parquet(outputs.df_X_train, mode='overwrite')
y_train.write.parquet(outputs.df_Y_train, mode='overwrite')

x_eval.write.parquet(outputs.df_X_eval, mode='overwrite')
y_eval.write.parquet(outputs.df_Y_eval, mode='overwrite')

x_test.write.parquet(outputs.df_X_test, mode='overwrite')
y_test.write.parquet(outputs.df_Y_test, mode='overwrite')

In [ ]:
#8 stop spark
SinaraSpark.stop_session()